#### REAL TIME DETECTION

In [1]:
import numpy as np
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [2]:
#loading the model and defining our region of interest 

model = keras.models.load_model(r"C:\Users\sweth\OneDrive\Desktop\final_model_review.h5")

num=[]
for i in range(1,10):
    num.append(i)

def listAlphabet():
  return [chr(i) for i in range(ord('A'),ord('Z')+1)]
alpha=listAlphabet()

classes=num+alpha

result = zip(range(35),classes)
mapping=dict(result)

background = None
accumulated_weight = 0.5
ROI_top = 100
ROI_bottom = 300
ROI_right = 150
ROI_left = 350

In [3]:
def cal_accum_avg(frame, accumulated_weight):

    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None

    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [4]:
#skin segmentation

def segment_hand(frame):
    
    
    grayImage = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Convert from RGB to HSV
    HSVImage = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) 

    # Finding pixels with itensity of skin
    lowerBoundary = np.array([0,40,30],dtype="uint8")
    upperBoundary = np.array([43,255,254],dtype="uint8")
    skinMask = cv2.inRange(HSVImage, lowerBoundary, upperBoundary)
    
    # blurring of gray scale using medianBlur
    skinMask = cv2.addWeighted(skinMask,0.5,skinMask,0.5,0.0)
    skinMask = cv2.medianBlur(skinMask, 5)
    skin = cv2.bitwise_and(grayImage, grayImage, mask = skinMask)
    
    _ , thresholded = cv2.threshold(skin, 22, 255,cv2.THRESH_BINARY)
    
    
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    # If length of contours list = 0, means we didn't get any contours
    if len(contours) == 0:
        return None
    else:
        # The largest external contour should be the hand 
        hand_segment_max_cont = max(contours, key=cv2.contourArea)
    
        return (thresholded, hand_segment_max_cont)

#### WEB CAM INTERFACE

In [5]:
cam = cv2.VideoCapture(0)
num_frames =0

while True:
     
    ret, frame = cam.read()

    # flipping the frame to prevent inverted image of captured frame
    
    frame = cv2.flip(frame, 1)

    frame_copy = frame.copy()

    # ROI from the frame
    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]

    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)
    

    if num_frames < 70:
        
        cal_accum_avg(gray_frame, accumulated_weight)
        
        cv2.putText(frame_copy, "FETCHING BACKGROUND...PLEASE WAIT",(80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    
    else: 
        # segmenting the hand region
        hand = segment_hand(roi)
        
        # Checking if we are able to detect the hand...
        if hand is not None:
            
            thresholded, hand_segment = hand

            # Drawing contours around hand segment
            cv2.drawContours(frame_copy, [hand_segment + (ROI_right,ROI_top)], -1, (255, 0, 0),1)
            
            cv2.imshow("Thesholded Hand Image", thresholded)
            
            thresholded = cv2.resize(thresholded, (128, 128))
            thresholded = np.reshape(thresholded,(1,thresholded.shape[0],thresholded.shape[1],1))
            
            pred = model.predict(thresholded)
            cv2.putText(frame_copy, str(mapping[np.argmax(pred)]), (170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
    # Draw ROI on frame_copy
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right,ROI_bottom), (255,128,0), 3)

    # incrementing the number of frames for tracking
    num_frames += 1

    # Display the frame with segmented hand
    cv2.putText(frame_copy, "ISL Recognition",(10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    cv2.imshow("Sign Detection", frame_copy)


    # Close windows with Esc
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

# Release the camera and destroy all the windows
cam.release()
cv2.destroyAllWindows()

In [29]:
cam.release()
cv2.destroyAllWindows()